In [1]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
from shapely.geometry import mapping
from tqdm import tqdm

In [ ]:
#load hex grid with EPSG:3067
hex_grid = gpd.read_parquet("../data/processed/hex_features.parquet")

hex_grid = hex_grid.to_crs("EPSG:4326")

In [ ]:
raster = "../data/raw/fin_ppp_2020.tif"

In [ ]:
with rasterio.open(raster) as src:

    if src.crs.to_string() != hex_grid.crs.to_string():
        print("CRS mismatch, fix this before running.")

    pop_values = []

    for geom in tqdm(hex_grid.geometry, desc="Processing Hexes"):

        geo = [mapping(geom)]

        try:
           out_image, out_transform = rasterio.mask.mask(src, geo, crop=True)
        except ValueError:
            pop_values.append(0)
            continue

        out_image = np.where(out_image == src.nodata, np.nan, out_image)

        pop_sum = np.nansum(out_image)
        pop_values.append(pop_sum)

hex_grid["population"] = pop_values

hex_grid.to_parquet("../data/processed/hex_with_population.parquet")

Processing Hexes: 100%|██████████████████████████████████████████████████████████| 4790/4790 [00:02<00:00, 2260.68it/s]


In [23]:

raster_path = "../data/raw/fin_ppp_2020.tif"

with rasterio.open(raster_path) as src:
    print("Raster CRS:", src.crs)


Raster CRS: EPSG:4326


In [25]:
hex_grid["population"].describe()

count    4790.000000
mean      508.101990
std       898.347107
min         0.000000
25%         0.000000
50%         0.000000
75%       608.559753
max      3032.876465
Name: population, dtype: float64